# BPO

In [3]:
from koselleck import *

In [4]:
# get_numlines??

In [14]:

def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])

#     odf['num_lines']=odf.path.progress_apply(get_numlines)
#     odf['num_words']=odf['num_lines']*skipgram_n
    return odf#.query('1700<=year')

In [15]:
dfskip=get_skipgrams()
dfskip

,corpus,year,path
182,bpo,1681,/home/ryan/github/koselleck/data/skipgrams/yea...
56,bpo,1682,/home/ryan/github/koselleck/data/skipgrams/yea...
51,bpo,1683,/home/ryan/github/koselleck/data/skipgrams/yea...
60,bpo,1684,/home/ryan/github/koselleck/data/skipgrams/yea...
200,bpo,1685,/home/ryan/github/koselleck/data/skipgrams/yea...
...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...
34,bpo,1969,/home/ryan/github/koselleck/data/skipgrams/yea...
87,bpo,1970,/home/ryan/github/koselleck/data/skipgrams/yea...


In [11]:
def periodize(y,ybin=10):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [12]:
periodize(1705)

'1700-1710'

In [13]:
dfskip['period']=dfskip.year.apply(periodize)
dfskip[~dfskip.period.isnull()]

,corpus,year,path,period
157,bpo,1700,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1710
42,bpo,1701,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1710
25,bpo,1702,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1710
288,bpo,1703,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1710
69,bpo,1704,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1710
...,...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...,1960-1970
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...,1960-1970
34,bpo,1969,/home/ryan/github/koselleck/data/skipgrams/yea...,1960-1970
87,bpo,1970,/home/ryan/github/koselleck/data/skipgrams/yea...,1970-1980


In [12]:
dfskipruns=pd.concat([
    dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
    for i in range(25)
])
dfskipruns

,corpus,year,path,period,run
157,bpo,1700,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770,run_01
42,bpo,1701,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770,run_01
25,bpo,1702,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770,run_01
288,bpo,1703,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770,run_01
69,bpo,1704,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770,run_01
...,...,...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
34,bpo,1969,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
87,bpo,1970,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25


In [15]:
# disable_gensim_logging()
enable_gensim_logging()

In [16]:
from gensim.models import FastText
from gensim.models import Word2Vec


def gen_and_save_model(dfskip, nskip=25000, force=False):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        model = Word2Vec(vector_size=300, window=10, min_count=50, sentences=ss, epochs=5, workers=8)
        model.save(ofnfn)
    return pd.DataFrame()

# model.wv.most_similar('value',topn=25)

In [18]:
dfskipdo=dfskipruns.query('period=="1830-1900"')
dfskipdo

,corpus,year,path,period,run
198,bpo,1830,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_01
220,bpo,1831,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_01
163,bpo,1832,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_01
269,bpo,1833,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_01
11,bpo,1834,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_01
...,...,...,...,...,...
21,bpo,1895,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
108,bpo,1896,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
17,bpo,1897,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
285,bpo,1898,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25


In [ ]:
pmap_groups(gen_and_save_model, dfskipdo.groupby(['period','run']), num_proc=4, kwargs=dict(force=True))

Mapping gen_and_save_model [x4]:   0%|          | 0/25 [00:00<?, ?it/s]collecting all words and their counts
collecting all words and their counts
collecting all words and their counts
collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 417319 words, keeping 27814 word types
PROGRESS: at sentence #10000, processed 417852 words, keeping 27870 word types
PROGRESS: at sentence #10000, processed 415969 words, keeping 27785 word types
PROGRESS: at sentence #10000, processed 416886 words, keeping 27780 word types
PROGRESS: at sentence #20000, processed 834354 words, keeping 37818 word types
PROGRESS: at sentence #20000, processed 827615 words, keeping 37564 word types
PROGRESS: at sentence #20000, processed

In [1]:
from gensim.models import KeyedVectors
m=KeyedVectors.load('/home/ryan/github/koselleck/data/models/bpo/1700-1770/run_01/model.bin')
m

In [2]:
m.wv.most_similar('history')

[('histories', 0.65504390001297),
 ('chronology', 0.6540281176567078),
 ('memoirs', 0.6262167692184448),
 ('annals', 0.6260145902633667),
 ('historical', 0.6221401691436768),
 ('antiquities', 0.6221285462379456),
 ('abridgment', 0.602946937084198),
 ('continuation', 0.5915682911872864),
 ('geography', 0.587739109992981),
 ('transactions', 0.577065110206604)]